In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from math import radians, cos, sin, asin, sqrt
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

## 數據縮減
## 離群值偵測

In [5]:
# 載入剛上傳的資料
df = pd.read_csv("Zomato Dataset.csv")

# 移除無用欄位 
df.drop(columns=['ID', 'Delivery_person_ID', 'Vehicle_condition'], inplace=True)

# 修正異常經緯度
df['Restaurant_latitude'] = df['Restaurant_latitude'].abs()
df['Delivery_location_latitude'] = df['Delivery_location_latitude'].abs()
df['Restaurant_longitude'] = df['Restaurant_longitude'].abs()
df['Delivery_location_longitude'] = df['Delivery_location_longitude'].abs()

# 數據縮減：標記欄位資訊（常數欄、低變異欄、唯一欄）
reduction_info = []

for col in df.columns:
    unique_vals = df[col].nunique(dropna=False)
    missing_pct = df[col].isnull().mean() * 100
    dtype = df[col].dtype
    if unique_vals == 1:
        reason = "常數欄（無變化）"
    elif unique_vals == len(df):
        reason = "唯一值欄（如 ID）"
    elif df[col].value_counts(normalize=True, dropna=False).iloc[0] > 0.98:
        reason = "主類佔比超過98%"  # 幾乎全都是某一類
    else:
        reason = ""
    reduction_info.append({
        'Column': col,
        'Data Type': dtype,
        'Unique Values': unique_vals,
        'Missing %': missing_pct,
        'Reduction Suggestion': reason
    })

reduction_df = pd.DataFrame(reduction_info)

# 離群值偵測（只針對數值欄）
numerical_columns = df.select_dtypes(include='number').columns
outlier_summary = []

for col in numerical_columns:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    total = len(df)
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    outlier_count = len(outliers)
    outlier_pct = outlier_count / total * 100
    outlier_summary.append({
        'Column': col,
        'Q1': q1,
        'Q3': q3,
        'IQR': iqr,
        'Lower Bound': lower,
        'Upper Bound': upper,
        'Outlier Count': outlier_count,
        'Outlier %': outlier_pct
    })

outlier_df = pd.DataFrame(outlier_summary)

In [7]:
reduction_df

,Column,Data Type,Unique Values,Missing %,Reduction Suggestion
0,Delivery_person_Age,float64,23,4.067217,
1,Delivery_person_Ratings,float64,29,4.185679,
2,Restaurant_latitude,float64,389,0.000000,
3,Restaurant_longitude,float64,389,0.000000,
4,Delivery_location_latitude,float64,4373,0.000000,
5,Delivery_location_longitude,float64,4373,0.000000,
6,Order_Date,object,44,0.000000,
7,Time_Orderd,object,177,3.797385,
8,Time_Order_picked,object,193,0.000000,
9,Weather_conditions,object,7,1.351351,


In [9]:
outlier_df

,Column,Q1,Q3,IQR,Lower Bound,Upper Bound,Outlier Count,Outlier %
0,Delivery_person_Age,25.000000,35.000000,10.000000,10.000000,50.000000,0,0.000000
1,Delivery_person_Ratings,4.500000,4.900000,0.400000,3.900000,5.500000,1223,2.682959
2,Restaurant_latitude,12.934179,22.732225,9.798046,-1.762890,37.429294,0,0.000000
3,Restaurant_longitude,73.170283,78.046106,4.875823,65.856549,85.359840,4497,9.865304
4,Delivery_location_latitude,12.988453,22.785049,9.796596,-1.706441,37.479943,0,0.000000
5,Delivery_location_longitude,73.280000,78.107044,4.827044,66.039434,85.347610,6515,14.292296
6,multiple_deliveries,0.000000,1.000000,1.000000,-1.500000,2.500000,361,0.791945
7,Time_taken (min),19.000000,32.000000,13.000000,-0.500000,51.500000,270,0.592313


## 刪掉經緯度異常

In [12]:
df = df[
    (df['Restaurant_latitude'] > 5) & (df['Restaurant_latitude'] < 40) &
    (df['Restaurant_longitude'] > 65) & (df['Restaurant_longitude'] < 100) &
    (df['Delivery_location_latitude'] > 5) & (df['Delivery_location_latitude'] < 40) &
    (df['Delivery_location_longitude'] > 65) & (df['Delivery_location_longitude'] < 100)
]

## 統計缺失情況

In [14]:
# 統計缺失情況
missing_stats = df.isnull().sum().to_frame(name="Missing Count")
missing_stats["Total"] = len(df)
missing_stats["Missing %"] = (missing_stats["Missing Count"] / missing_stats["Total"]) * 100
missing_stats["Action Suggestion"] = missing_stats["Missing %"].apply(
    lambda x: "Drop Column" if x > 40 else "Impute"
)

# 排序顯示
missing_stats = missing_stats.sort_values(by="Missing %", ascending=False)
missing_stats

,Missing Count,Total,Missing %,Action Suggestion
Delivery_person_Ratings,1763,41944,4.203223,Impute
Delivery_person_Age,1719,41944,4.098322,Impute
Time_Orderd,1600,41944,3.814610,Impute
City,1114,41944,2.655922,Impute
multiple_deliveries,905,41944,2.157639,Impute
Weather_conditions,569,41944,1.356571,Impute
Road_traffic_density,555,41944,1.323193,Impute
Festival,215,41944,0.512588,Impute
Type_of_vehicle,0,41944,0.000000,Impute
Type_of_order,0,41944,0.000000,Impute


## Delivery_person_Ratings	的補值
- 優先用同樣 City + Type_of_vehicle 的平均值，否則補整體平均

In [17]:
# 對每個 (City, Type_of_vehicle) 群組計算平均評分
grouped_rating = df.groupby(['City', 'Type_of_vehicle'])['Delivery_person_Ratings'].mean()

# 定義補值函數：優先用 (City, Type_of_vehicle) 平均值，否則補整體平均
overall_mean_rating = df['Delivery_person_Ratings'].mean()

def fill_rating(row):
    if pd.notnull(row['Delivery_person_Ratings']):
        return row['Delivery_person_Ratings']
    group_key = (row['City'], row['Type_of_vehicle'])
    if group_key in grouped_rating:
        return grouped_rating[group_key]
    else:
        return overall_mean_rating

# 補值
df['Delivery_person_Ratings'] = df.apply(fill_rating, axis=1)

# 檢查補值後是否還有缺
missing_after = df['Delivery_person_Ratings'].isnull().sum()
missing_after

12

In [19]:
# 直接刪除這些補不到的資料（還是 NaN）
df = df[df['Delivery_person_Ratings'].notnull()]

# 確認是否完全沒有缺值
df['Delivery_person_Ratings'].isnull().sum()

0

## Delivery_person_Age 的補值
- 用相同 City + Type_of_vehicle 分群的中位數

In [21]:
# 對每個 (City, Type_of_vehicle) 群組計算中位數年齡
grouped_age = df.groupby(['City', 'Type_of_vehicle'])['Delivery_person_Age'].median()

# 全體中位數備用
overall_median_age = df['Delivery_person_Age'].median()

# 補值函數
def fill_age(row):
    if pd.notnull(row['Delivery_person_Age']):
        return row['Delivery_person_Age']
    group_key = (row['City'], row['Type_of_vehicle'])
    if group_key in grouped_age:
        return grouped_age[group_key]
    else:
        return overall_median_age

# 執行補值
df['Delivery_person_Age'] = df.apply(fill_age, axis=1)

# 檢查是否補完
missing_age_after = df['Delivery_person_Age'].isnull().sum()
missing_age_after

0

## Time_Orderd的補值
- 如果 Time_Order_picked 有值，可以用它減去備餐平均時間來回推 Time_Orderd
- 如果兩欄都沒值 → 直接刪除該筆資料

In [23]:
# 補值前先處理 Time_Orderd 和 Time_Order_picked 欄位的格式
def clean_excel_time(val):
    if pd.isnull(val):  # 真正的 NaN
        return None
    try:
        if isinstance(val, (float, int)):
            time_val = float(val)
        elif isinstance(val, str):
            val = val.strip().lower()
            if val in ['nan', '', 'none']:
                return None
            time_val = float(val) if '.' in val else None
            if time_val is None:
                return val  # 留下正常時間字串
        else:
            return None

        # 將小數轉成時間
        total_minutes = int(round(time_val * 24 * 60))
        hours = total_minutes // 60
        minutes = total_minutes % 60
        return f"{hours:02d}:{minutes:02d}"
    except:
        return None

# 應用於兩欄
df['Time_Orderd'] = df['Time_Orderd'].apply(clean_excel_time)
df['Time_Order_picked'] = df['Time_Order_picked'].apply(clean_excel_time)

# 計算平均備餐時間（分鐘）
valid_prep_time = pd.to_datetime(df['Time_Order_picked'], format="%H:%M", errors='coerce') - \
                  pd.to_datetime(df['Time_Orderd'], format="%H:%M", errors='coerce')
prep_time_avg_min = valid_prep_time.dropna().dt.total_seconds().mean() / 60

# 刪掉 Time_Order_picked 轉換失敗的資料
df = df[pd.to_datetime(df['Time_Order_picked'], format="%H:%M", errors='coerce').notnull()]

# 補回缺 Time_Orderd 的資料
def estimate_time_orderd(row):
    if pd.notnull(row['Time_Orderd']):
        return row['Time_Orderd']
    if pd.notnull(row['Time_Order_picked']):
        try:
            pickup_time = pd.to_datetime(row['Time_Order_picked'], format="%H:%M")
            estimated_time = pickup_time - pd.to_timedelta(prep_time_avg_min, unit='m')
            return estimated_time.strftime("%H:%M")
        except:
            return None
    return None

df['Time_Orderd'] = df.apply(estimate_time_orderd, axis=1)

# 刪除兩欄皆空值的資料
df = df[~(df['Time_Orderd'].isnull() & df['Time_Order_picked'].isnull())]

# 檢查結果
missing_Time_Orderd_after = df['Time_Orderd'].isnull().sum()
missing_Time_Orderd_after


0

## City 的補值
- 把所有有 `City` 的資料轉成經緯度座標表
- 對每一筆 `City` 缺的資料，用最近鄰居的城市補上
- 用 KDTree 快速查最鄰近點 

In [26]:
# 分出有 City 的資料（作為參考點）與沒有 City 的資料（要補）
city_known = df[df['City'].notnull()]
city_missing = df[df['City'].isnull()]

# 準備 KDTree（用餐廳的經緯度）
known_coords = city_known[['Restaurant_latitude', 'Restaurant_longitude']].values
missing_coords = city_missing[['Restaurant_latitude', 'Restaurant_longitude']].values

tree = KDTree(known_coords)

# 找最近鄰，並用鄰居的 City 來補
_, indices = tree.query(missing_coords, k=1)
nearest_city_values = city_known.iloc[indices.flatten()]['City'].values

# 寫回缺失的 City
df.loc[df['City'].isnull(), 'City'] = nearest_city_values

# 檢查還有沒有缺值
df['City'].isnull().sum()

0

## multiple_deliveries 的補值 
- 將原先的補值方式更新為缺值全刪除
- 暫時保留全部NaN
- 後續過程中會刪除

In [29]:
# 檢查NaN數量
df['multiple_deliveries'].isnull().sum()

884

## Weather_conditions 的補值
- 用「時間 ± 1 小時」＋「地點最近的 5 筆資料」來找出最可能的天氣條件，接著取眾數補上

In [31]:
# 建立完整的下單時間欄位
df['order_datetime'] = pd.to_datetime(df['Order_Date'] + ' ' + df['Time_Orderd'], format='%d-%m-%Y %H:%M', errors='coerce')

# 分出有天氣和沒有天氣的資料
df_weather_known = df[df['Weather_conditions'].notnull()].copy()
df_weather_missing = df[df['Weather_conditions'].isnull()].copy()

# 計算地理距離（Haversine formula）
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

# 預補欄位
estimated_weather = []

# 依序處理每筆缺天氣的資料
for idx, row in df_weather_missing.iterrows():
    lat, lon, time = row['Restaurant_latitude'], row['Restaurant_longitude'], row['order_datetime']
    if pd.isnull(lat) or pd.isnull(lon) or pd.isnull(time):
        estimated_weather.append(None)
        continue

    # 找在時間 ± 1小時 內的資料
    time_window = (df_weather_known['order_datetime'] >= time - timedelta(minutes=60)) & \
                  (df_weather_known['order_datetime'] <= time + timedelta(minutes=60))

    # 找出符合時間條件的子集
    nearby = df_weather_known[time_window].copy()
    if nearby.empty:
        estimated_weather.append(None)
        continue

    # 計算這些資料與該缺失點的距離
    nearby['distance'] = nearby.apply(
        lambda r: haversine(lat, lon, r['Restaurant_latitude'], r['Restaurant_longitude']), axis=1)

    # 找距離最小的5筆，取天氣的眾數補值
    nearest = nearby.nsmallest(5, 'distance')
    mode_weather = nearest['Weather_conditions'].mode()
    estimated_weather.append(mode_weather[0] if not mode_weather.empty else None)

# 寫回資料
df.loc[df['Weather_conditions'].isnull(), 'Weather_conditions'] = estimated_weather

# 檢查補完後還剩幾筆 NaN
df['Weather_conditions'].isnull().sum()

0

## Road_traffic_density 的補值
- 用「時間 ± 1 小時」＋「地點最近的 5 筆資料」來找出最可能的路況，接著取眾數補上

In [33]:
# 分出有 traffic 和沒有 traffic 的資料
df_traffic_known = df[df['Road_traffic_density'].notnull()].copy()
df_traffic_missing = df[df['Road_traffic_density'].isnull()].copy()

# 預補欄位
estimated_traffic = []

# 依序處理每筆缺交通的資料
for idx, row in df_traffic_missing.iterrows():
    lat, lon, time = row['Restaurant_latitude'], row['Restaurant_longitude'], row['order_datetime']
    if pd.isnull(lat) or pd.isnull(lon) or pd.isnull(time):
        estimated_traffic.append(None)
        continue

    # 找在時間 ± 1小時 內的資料
    time_window = (df_traffic_known['order_datetime'] >= time - timedelta(minutes=60)) & \
                  (df_traffic_known['order_datetime'] <= time + timedelta(minutes=60))

    nearby = df_traffic_known[time_window].copy()
    if nearby.empty:
        estimated_traffic.append(None)
        continue

    nearby['distance'] = nearby.apply(
        lambda r: haversine(lat, lon, r['Restaurant_latitude'], r['Restaurant_longitude']), axis=1)

    nearest = nearby.nsmallest(5, 'distance')
    mode_traffic = nearest['Road_traffic_density'].mode()
    estimated_traffic.append(mode_traffic[0] if not mode_traffic.empty else None)

# 寫回資料
df.loc[df['Road_traffic_density'].isnull(), 'Road_traffic_density'] = estimated_traffic

# 檢查補完後還剩幾筆 NaN
df['Road_traffic_density'].isnull().sum()

0

## Festival 的補值
- 將原先的補值方式更新為缺值以及Yes全刪除
- 
暫時保留全部Na
- 
後續過程中會No

In [37]:
# 檢查NaN數量
df['Festival'].isnull().sum()

204

In [39]:
# 統計缺失情況
missing_stats = df.isnull().sum().to_frame(name="Missing Count")
missing_stats["Total"] = len(df)
missing_stats["Missing %"] = (missing_stats["Missing Count"] / missing_stats["Total"]) * 100
missing_stats["Action Suggestion"] = missing_stats["Missing %"].apply(
    lambda x: "Drop Column" if x > 40 else "Impute"
)

# 排序顯示
missing_stats = missing_stats.sort_values(by="Missing %", ascending=False)
missing_stats

,Missing Count,Total,Missing %,Action Suggestion
multiple_deliveries,884,40721,2.17087,Impute
Festival,204,40721,0.50097,Impute
Delivery_person_Age,0,40721,0.00000,Impute
Delivery_person_Ratings,0,40721,0.00000,Impute
Time_taken (min),0,40721,0.00000,Impute
City,0,40721,0.00000,Impute
Type_of_vehicle,0,40721,0.00000,Impute
Type_of_order,0,40721,0.00000,Impute
Road_traffic_density,0,40721,0.00000,Impute
Weather_conditions,0,40721,0.00000,Impute


## 新增直線距離欄位

In [42]:
#  distance_km 欄位 兩點直線距離
df['distance_km'] = df.apply(lambda row: haversine(
    row['Restaurant_latitude'], row['Restaurant_longitude'],
    row['Delivery_location_latitude'], row['Delivery_location_longitude']
), axis=1)

In [44]:
df

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),order_datetime,distance_km
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,Snack,motorcycle,3.0,No,Metropolitian,46,2022-02-12 21:55:00,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,Meal,motorcycle,1.0,No,Metropolitian,23,2022-02-13 14:55:00,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,Drinks,scooter,1.0,No,Metropolitian,21,2022-03-04 17:30:00,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,Buffet,motorcycle,0.0,No,Metropolitian,20,2022-02-13 09:20:00,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,Snack,scooter,1.0,No,Metropolitian,41,2022-02-14 19:50:00,19.396618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45577,28.0,4.9,13.029198,77.570997,13.059198,77.600997,30-03-2022,21:55,22:00,Sandstorms,Jam,Meal,scooter,1.0,No,Metropolitian,29,2022-03-30 21:55:00,4.657133
45578,35.0,4.2,23.371292,85.327872,23.481292,85.437872,08-03-2022,21:45,21:55,Windy,Jam,Drinks,motorcycle,1.0,No,Metropolitian,33,2022-03-08 21:45:00,16.600272
45579,30.0,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,Meal,motorcycle,0.0,No,Metropolitian,32,2022-03-24 11:35:00,1.489846
45582,20.0,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,Snack,motorcycle,1.0,No,Metropolitian,26,2022-03-07 13:35:00,6.232393


In [46]:
# 匯出結果
output_path = "zomato_with_dir_distance.csv"
df.to_csv(output_path, index=False)

# 更新
- 將API 計算出的路程耗時結果合併進入表單

In [49]:
# 讀取兩張表
df_duration = pd.read_csv("zomato_with_estimated_durations.csv")
df_dir_distance = pd.read_csv("zomato_with_dir_distance.csv")

# 加欄位，不動其他資料（假設兩個資料順序對得上）
df_dir_distance['Estimated_duration_minutes'] = df_duration['Estimated_duration_minutes']

# 儲存新檔
df_dir_distance.to_csv("zomato_combined.csv", index=False)